<a href="https://colab.research.google.com/github/Saronzeleke/Splitter/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!sudo apt-get install portaudio19-dev python-all-dev python3-all-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0 libpython-all-dev libpython2-dev
  libpython2-stdlib libpython2.7 libpython2.7-dev libpython2.7-minimal
  libpython2.7-stdlib libpython3-all-dev python-all python2 python2-dev
  python2-minimal python2.7 python2.7-dev python2.7-minimal python3-all
Suggested packages:
  portaudio19-doc python2-doc python-tk python2.7-doc binfmt-support
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 libpython-all-dev libpython2-dev
  libpython2-stdlib libpython2.7 libpython2.7-dev libpython2.7-minimal
  libpython2.7-stdlib libpython3-all-dev portaudio19-dev python-all
  python-all-dev python2 python2-dev python2-minimal python2.7 python2.7-dev
  python2.7-minimal python3-all python3-all-dev
0 upgraded, 21 newly installed, 0 to remove and 34 not upgraded.
Need to get 8,178 kB of archives.
After

In [10]:
!pip install pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp311-cp311-linux_x86_64.whl size=67425 sha256=6d8d1d9f93e42ac8ad63554c59b516b1486d2884167d115b9fd611ffeaf7caa0
  Stored in directory: /root/.cache/pip/wheels/80/b1/c1/67e4ef443de2665d86031d4760508094eab5de37d5d64d9c27
Successfully built pyaudio


In [11]:
!pip install keyboard

In [12]:
import pyaudio
import wave
import numpy as np
import keyboard
from threading import Thread, Event
import time
import os
from collections import deque
import json
from datetime import datetime
import argparse

In [15]:
class AudioSplitter:
    def __init__(self, silence_threshold=500, min_keystroke_gap=0.1, output_dir="split-audio", log_file="keystroke-log.json"):
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = 1
        self.RATE = 44100
        self.CHUNK = 1024
        self.SILENCE_THRESHOLD = silence_threshold
        self.MIN_KEYSTROKE_GAP = min_keystroke_gap

        self.audio = pyaudio.PyAudio()
        self.frames = []
        self.is_recording = False
        self.stop_event = Event()
        self.keystroke_times = []
        self.actual_keystrokes = []
        self.output_dir = output_dir
        self.log_file = log_file

        self.amplitude_window = deque(maxlen=5)
        self.last_keystroke_time = 0

        os.makedirs(self.output_dir, exist_ok=True)


In [16]:
def callback(self, in_data, frame_count, time_info, status):
        if self.is_recording:
            self.frames.append(in_data)
            audio_data = np.frombuffer(in_data, dtype=np.int16)
            current_amplitude = np.abs(audio_data).mean()
            self.amplitude_window.append(current_amplitude)

            if (len(self.amplitude_window) == self.amplitude_window.maxlen and
                current_amplitude > self.SILENCE_THRESHOLD and
                (time.time() - self.last_keystroke_time) > self.MIN_KEYSTROKE_GAP):
                self.last_keystroke_time = time.time()
                self.keystroke_times.append(len(self.frames) - 1)
        return (in_data, pyaudio.paContinue)


In [19]:
def start_recording(self):
        self.stream = self.audio.open(
            format=self.FORMAT,
            channels=self.CHANNELS,
            rate=self.RATE,
            input=True,
            frames_per_buffer=self.CHUNK,
            stream_callback=self.callback
        )
        self.is_recording = True
        print("Recording started... Press ESC to stop.")

In [20]:
def stop_recording(self):
        self.is_recording = False
        if hasattr(self, 'stream'):
            self.stream.stop_stream()
            self.stream.close()
        self.audio.terminate()
        print("Recording stopped.")

In [21]:
def save_audio_segment(self, filename, start_frame, end_frame):
        wf = wave.open(filename, 'wb')
        wf.setnchannels(self.CHANNELS)
        wf.setsampwidth(self.audio.get_sample_size(self.FORMAT))
        wf.setframerate(self.RATE)
        wf.writeframes(b''.join(self.frames[start_frame:end_frame]))
        wf.close()

In [22]:
def split_by_keystrokes(self):
        if not self.keystroke_times:
            print("No keystrokes detected")
            return
        split_points = [0] + self.keystroke_times + [len(self.frames)]
        for i in range(1, len(split_points)):
            start = split_points[i-1]
            end = split_points[i]
            filename = os.path.join(self.output_dir, f"segment_{i}.wav")
            self.save_audio_segment(filename, start, end)
            print(f"Saved {filename} (frames {start}-{end})")

In [23]:
def log_actual_keystroke(self, event):
        if event.event_type == keyboard.KEY_DOWN:
            timestamp = datetime.now().isoformat()
            self.actual_keystrokes.append({
                'key': event.name,
                'time': timestamp,
                'frame_index': len(self.frames) if self.is_recording else None
            })
            if len(self.actual_keystrokes) % 5 == 0:
                self.save_keystroke_log()

In [24]:
def save_keystroke_log(self):
        with open(self.log_file, 'w') as f:
            json.dump(self.actual_keystrokes, f, indent=2)



In [25]:
def monitor_keys(self):
        keyboard.hook(self.log_actual_keystroke)
        while not self.stop_event.is_set():
            time.sleep(0.1)
        keyboard.unhook_all()

In [26]:
def run(self):
        print(f"Amplitude threshold: {self.SILENCE_THRESHOLD}")
        print("Press ESC to stop and split by detected keystrokes")

        key_thread = Thread(target=self.monitor_keys)
        key_thread.start()
        self.start_recording()
        keyboard.wait('esc')
        self.stop_event.set()
        self.stop_recording()
        key_thread.join()
        print(f"\nDetected {len(self.keystroke_times)} keystrokes by amplitude")
        self.split_by_keystrokes()
        self.save_keystroke_log()
        print(f"\nActual keystrokes logged to {self.log_file}")
        print(f"Audio segments saved to '{self.output_dir}' directory")

In [37]:
if __name__ == "__main__":
    # Instead of parsing arguments, directly set the values:
    silence_threshold = 500  # Default value
    min_keystroke_gap = 0.1  # Default value
    output_dir = "split_audio"  # Default value
    log_file = "keystroke_log.json"  # Default value

    try:
        splitter = AudioSplitter(
            silence_threshold=silence_threshold,
            min_keystroke_gap=min_keystroke_gap,
            output_dir=output_dir,
            log_file=log_file
        )
        splitter.run()
    except Exception as e:
        import traceback
        traceback.print_exc()  # Print the full traceback for more details
        print(f"Error: {e}")

Error: 'AudioSplitter' object has no attribute 'run'


Traceback (most recent call last):
  File "<ipython-input-37-818d74d2ea6b>", line 15, in <cell line: 0>
    splitter.run()
    ^^^^^^^^^^^^
AttributeError: 'AudioSplitter' object has no attribute 'run'
